In [1]:
import os
import numpy as np
import pandas as pd
import supervision as sv
from supervision.metrics import MeanAveragePrecision

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

classes= ['CFCBK', 'FCBK', 'Zigzag']
import os
import re

In [2]:
import os
import re
import pandas as pd
# "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/mmrotate/configs/rotated_imted/rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h.py"

config="rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h"
# Path to your config file
config_file_path = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/mmrotate/configs/rotated_imted/rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h.py"

# Read config file
with open(config_file_path, 'r') as file:
    config_text = file.read()

# Extract all `type=...` assignments (handles both single and double quotes)
type_matches = re.findall(r"type\s*=\s*['\"]([^'\"]+)['\"]", config_text)

# Remove duplicates (optional)
type_matches = list(set(type_matches))

# Convert to DataFrame
df = pd.DataFrame(type_matches, columns=["type"])

# Show or save
print(df)
# df.to_csv("config_types.csv", index=False)


                           type
0                    Pretrained
1          DeltaXYWHAHBBoxCoder
2                         AdamW
3             LoadImageFromFile
4           DefaultFormatBundle
5                           Pad
6              EpochBasedRunner
7               AnchorGenerator
8                  SmoothL1Loss
9            RotatedMAEBBoxHead
10                      RResize
11           DeltaXYWHBBoxCoder
12             CrossEntropyLoss
13           SingleRoIExtractor
14              LoadAnnotations
15                     RoIAlign
16  RotatedStandardRoIHeadimTED
17                          nms
18                    Normalize
19                 RotatedimTED
20                      Collect
21               RotatedRPNHead
22               MaxIoUAssigner
23                  RRandomFlip
24                RandomSampler
25                    SimpleFPN
26                           LN
27            VisionTransformer


### Use that code 

In [3]:
def get_image_names_from_directory(directory):
    """Extracts image names (without extension) from a directory."""
    return {file_name.replace(".txt", "") for file_name in os.listdir(directory) if file_name.endswith(".txt")}

def load_detections(annotations_path, img_names, is_gt=True, confidence_threshold=0):
    """Loads detections only for images that exist in both GT and Predictions."""
    sv_data = []

    for image_id in sorted(img_names):
        file_path = os.path.join(annotations_path, f"{image_id}.txt")
        if not os.path.exists(file_path):  # Ensure file exists before processing
            continue

        xyxy_list = []
        class_ids = []
        scores = []

        with open(file_path, "r") as file:
            lines = file.readlines()

        if not lines:
            detection = sv.Detections(
                xyxy=np.empty((0, 4)),
                class_id=np.empty((0,)),
                confidence=np.empty((0,)),
                # metadata={"image_id": image_id}
            )
            sv_data.append(detection)
            continue
        
        for line in lines:
            data = line.strip().split(" ")
            class_name = data[8]
            class_id = classes.index(class_name) if class_name in classes else -1
            if class_id == -1:
                continue
            polygon = np.array(list(map(float, data[:8]))).reshape(4, 2)  # Convert to (4,2) shape with floats
            score = float(data[9]) if not is_gt else 1.0  # Default confidence for GT is 1.0

            if not is_gt and score < confidence_threshold:
                continue

            # Convert quadrilateral to bounding box (min x, min y, max x, max y)
            x_min, y_min = np.min(polygon, axis=0)
            x_max, y_max = np.max(polygon, axis=0)
            bbox = [x_min, y_min, x_max, y_max]

            # Append to lists
            xyxy_list.append(bbox)
            class_ids.append(class_id)
            scores.append(score)

        # Convert lists into a Supervision Detections object
        detections = sv.Detections(
            xyxy=np.array(xyxy_list),
            class_id=np.array(class_ids),
            confidence=np.array(scores),
            # metadata={"image_id": image_id}
        )

        sv_data.append(detections)

    return sv_data

def get_class_counts(detections_list, num_classes=3):
    """Counts occurrences of each class in ground truth detections."""
    class_counts = np.zeros(num_classes)
    for detections in detections_list:
        unique, counts = np.unique(detections.class_id, return_counts=True)
        for cls, count in zip(unique, counts):
            # print(cls, count)
            class_counts[cls] += count
    return class_counts


In [ ]:


# .py
def extract_model_structure(config_path):
    with open(config_path, 'r') as f:
        content = f.read()

    return {
        "model_type": re.search(r"model\s*=\s*dict\([\s\S]*?type=['\"](.+?)['\"]", content).group(1) if re.search(r"model\s*=", content) else None,
        "backbone_type": re.search(r"backbone\s*=\s*dict\([\s\S]*?type=['\"](.+?)['\"]", content).group(1) if re.search(r"backbone\s*=", content) else None,
        "neck_type": re.search(r"neck\s*=\s*dict\([\s\S]*?type=['\"](.+?)['\"]", content).group(1) if re.search(r"neck\s*=", content) else None,
        "bbox_head_type": re.search(r"bbox_head\s*=\s*dict\([\s\S]*?type=['\"](.+?)['\"]", content).group(1) if re.search(r"bbox_head\s*=", content) else None,
    }

# # === CONFIGURATIONS === #
# Base paths
base_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/mmrotate"
data_base_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/jeet/test/HRDet/data/dota"
data_type = 'test'
inference_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/results/rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h_nms_0.5_conf_0.1"
working_dir_name = "work_dirs_saved"

# Define config_name separately to use it in config_file and inference_dir
config_name = 'rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h'
model_name = 'rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h'

# Path to the config file
# config_file_path = os.path.join(base_path, f"configs/{model_name}/{config_name}.py")

print(f"Config file path: {config_file_path}")



model_info = extract_model_structure(config_file_path)

model_configs = [
    {
        'model_name': model_name,
        'train_data': "stratified_train",
        'test_data': "stratified_test",
        'config_name': config_name,
        'config_file': config_file_path,
        # 'checkpoint_folder': os.path.join(base_path, f'{working_dir_name}/{model_name}'),
        'val_dir': os.path.join(data_base_path, 'test'),
        'inference_dir': os.path.join(inference_dir, f"test_{config_name}_nms_0.5_conf_0.1"),
        'epoch': 25,
        'conf_threshold': 0.05,
        **model_info
    }
]

# Validate paths
for cfg in model_configs:
    assert os.path.isfile(cfg['config_file']), f"Missing config file: {cfg['config_file']}"
    # assert os.path.isdir(cfg['checkpoint_folder']), f"Missing checkpoint folder: {cfg['checkpoint_folder']}"
    assert os.path.isdir(cfg['val_dir']), f"Missing validation directory: {cfg['val_dir']}"

# === RESULTS DF SETUP === #
columns = ['model_name', "CFCBK", "FCBK", "Zigzag", "Weighted mAP@50", "mAP@50:95", "mAP@50", "mAP@75",
           "CA mAP@50:95", "CA mAP@50", "CA mAP@75", "confidence_threshold",
           "backbone", "head", "epoch", "model_type", "backbone_type", "neck_type", "bbox_head_type"]

index = pd.MultiIndex.from_tuples([], names=["Backbone", "Head", "Epoch"])
result_df = pd.DataFrame(columns=columns, index=index)

# === EVALUATION === #
for cfg in model_configs:
    print(f"\nEvaluating: {cfg['config_name']}")

    GT_PATH = os.path.join(cfg['val_dir'], "labelTxt_224")
    print(f"GT_PATH: {GT_PATH}")
    # PRED_PATH = os.path.join(cfg['inference_dir'], f"epoch_{cfg['epoch']}", "annfiles")
    # print(f"PRED_PATH: {PRED_PATH}")
    PRED_PATH="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/results/rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h_nms_0.5_conf_0.1/epoch_25/v4/annfiles"

    assert os.path.isdir(GT_PATH), f"Missing GT path: {GT_PATH}"
    assert os.path.isdir(PRED_PATH), f"Missing Predictions: {PRED_PATH}"

    gt_imgs = get_image_names_from_directory(GT_PATH)
    pred_imgs = get_image_names_from_directory(PRED_PATH)
    common_imgs = gt_imgs.intersection(pred_imgs)
    assert len(common_imgs) > 0, "No common images to evaluate!"

    gt_data = load_detections(GT_PATH, common_imgs, is_gt=True)
    pred_data = load_detections(PRED_PATH, common_imgs, is_gt=False, confidence_threshold=cfg['conf_threshold'])

    # Class-wise mAP
    mAP = MeanAveragePrecision(class_agnostic=False)
    mAP_result = mAP.update(pred_data, gt_data).compute()

    matched = mAP_result.matched_classes.tolist()
    class_ap = mAP_result.ap_per_class[:, 0].tolist()
    class_ap_final = [0.0] * 3
    for cls, ap in zip(matched, class_ap):
        class_ap_final[cls] = ap

    counts = get_class_counts(gt_data, num_classes=3)
    weighted_ap = np.sum(np.array(class_ap_final) * counts) / np.sum(counts)

    # Class-agnostic mAP
    mAP_ca = MeanAveragePrecision(class_agnostic=True)
    mAP_result_ca = mAP_ca.update(pred_data, gt_data).compute()

    row_key = (cfg['backbone_type'], cfg['bbox_head_type'], cfg['epoch'])
    row_data = class_ap_final + [
        weighted_ap,
        mAP_result.map50_95, mAP_result.map50, mAP_result.map75,
        mAP_result_ca.map50_95, mAP_result_ca.map50, mAP_result_ca.map75,
        cfg['conf_threshold'],
        cfg['backbone_type'], cfg['bbox_head_type'], cfg['epoch'],
        cfg['model_type'], cfg['backbone_type'], cfg['neck_type'], cfg['bbox_head_type']
    ]

    result_df.loc[row_key] = [f"{v:.6f}" if isinstance(v, float) else v for v in [cfg['model_name']] + row_data]

#import os
import pandas as pd

# === SAVE RESULTS === #
base_path = ""
csv_path = os.path.join(base_path, "map_results_with_model_info.csv")

# Example: result_df should already be defined before this
# For demonstration, here's a dummy example:
# result_df = pd.DataFrame([{"model": "YOLOv8", "mAP": 0.72, "F1": 0.85}])

# Append new results to the existing CSV file
if os.path.exists(csv_path):
    old_df = pd.read_csv(csv_path)
    result_df = pd.concat([old_df, result_df], ignore_index=True)
else:
    print("CSV file does not exist. A new file will be created.")

# Save the updated DataFrame
result_df.to_csv(csv_path, index=False)

# Print the DataFrame and confirmation
print("\n=== Evaluation Results ===")
print(result_df)
print("\n✅ Evaluation complete. Results saved to 'map_results_with_model_info.csv'")


Config file path: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/STD/mmrotate/configs/rotated_imted/rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h.py

Evaluating: rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h
GT_PATH: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/jeet/test/HRDet/data/dota/test/labelTxt_224

=== Evaluation Results ===
                                          model_name     CFCBK      FCBK  \
0  rotated_imted_faster_rcnn_vit_small_1x_dota_le...  0.021833  0.269714   
1  rotated_imted_faster_rcnn_vit_small_1x_dota_le...  0.021865   0.26625   
2  rotated_imted_faster_rcnn_vit_small_1x_dota_le...       0.0       0.0   
3  rotated_imted_faster_rcnn_vit_small_1x_dota_le...  0.000000  0.121257   

     Zigzag Weighted mAP@50 mAP@50:95    mAP@50    mAP@75 CA mAP@50:95  \
0  0.257105        0.256523   0.05858  0.182884  0.019319     0.094055   
1  0.256751        0.254528  0.058297  0.181622  0.019312     0.093958   
2       0.0             0.0

In [6]:
display(result_df)

,model_name,CFCBK,FCBK,Zigzag,Weighted mAP@50,mAP@50:95,mAP@50,mAP@75,CA mAP@50:95,CA mAP@50,CA mAP@75,confidence_threshold,backbone,head,epoch,model_type,backbone_type,neck_type,bbox_head_type
0,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,0.021833,0.269714,0.257105,0.256523,0.05858,0.182884,0.019319,0.094055,0.291076,0.034251,0.05,VisionTransformer,RotatedMAEBBoxHead,25,RotatedimTED,VisionTransformer,SimpleFPN,RotatedMAEBBoxHead
1,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,0.021865,0.26625,0.256751,0.254528,0.058297,0.181622,0.019312,0.093958,0.291318,0.034258,0.05,VisionTransformer,RotatedMAEBBoxHead,25,RotatedimTED,VisionTransformer,SimpleFPN,RotatedMAEBBoxHead
2,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,VisionTransformer,RotatedMAEBBoxHead,25,RotatedimTED,VisionTransformer,SimpleFPN,RotatedMAEBBoxHead
3,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,0.000000,0.121257,0.083975,0.101070,0.017690,0.068411,0.000780,0.037770,0.150805,0.001864,0.050000,VisionTransformer,RotatedMAEBBoxHead,25,RotatedimTED,VisionTransformer,SimpleFPN,RotatedMAEBBoxHead


In [ ]:
csv="map_results_with_model_info.csv"
s=pd.read_csv(csv)


Append to save to csv file

In [ ]:
#take the columns [model_name CA mAP@50	CFCBK	FCBK	Zigzag	]
s=s[['model_name', 'CA mAP@50', 'CFCBK', 'FCBK', 'Zigzag']]
#calculate parcentage of each column 
s['CA mAP@50'] = (s['CA mAP@50'] * 100)
s['CFCBK'] = (s['CFCBK'] * 100)
s['FCBK'] = (s['FCBK'] * 100)
s['Zigzag'] = (s['Zigzag'] * 100)
#save the dataframe to csv
# s.to_csv(os.path.join(base_path, "map_results_with_model_info_percentage.csv"), index=False)
s

,model_name,CA mAP@50,CFCBK,FCBK,Zigzag
0,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,29.1076,2.1833,26.9714,25.7105
1,rotated_imted_faster_rcnn_vit_small_1x_dota_le...,29.1318,2.1865,26.6250,25.6751


In [ ]:

# csv_path = os.path.join(base_path, "map_results_with_model_info.csv")
s = s[['model_name', 'CA mAP@50', 'CFCBK', 'FCBK', 'Zigzag']]

# Convert values to percentages and round to 2 decimal places
for col in ['CA mAP@50', 'CFCBK', 'FCBK', 'Zigzag']:
    s[col] = (s[col]).round(2)

# Save the updated DataFrame to CSV
# s.to_csv(os.path.join(base_path, "map_results_with_model_info_percentage.csv"), index=False)

for _, row in s.iterrows():
    model_name = row['model_name']
    values = [f"{row[col]:.2f}" for col in ['CA mAP@50', 'CFCBK', 'FCBK', 'Zigzag']]
    formatted_row = f"{model_name} & " + " & ".join(values) + " \\\\"
    print(formatted_row)

rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h & 29.11 & 2.18 & 26.97 & 25.71 \\
rotated_imted_faster_rcnn_vit_small_1x_dota_le90_8h & 29.13 & 2.19 & 26.62 & 25.68 \\
